In [142]:
import requests
import json
from datetime import datetime
import pandas as pd

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAK24TQEAAAAAHG0C3s8WeHg3xoXvrJpFaKNXQXw%3DdmVYnBko1CicATtTRO1KP5DHZLZBwOF88NrO7ZH62K8ELY7QtL'


def create_url(user_id=1339835893, start_time='2016-01-01T00:00:00Z', end_time='2017-01-01T00:00:00Z'):
	# Replace with user ID below
	return 'https://api.twitter.com/2/users/{}/tweets?max_results=100&start_time={}&end_time={}'.format(user_id, start_time, end_time)


def get_params():
	# Tweet fields are adjustable.
	# Options include:
	# attachments, author_id, context_annotations,
	# conversation_id, created_at, entities, geo, id,
	# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
	# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
	# source, text, and withheld
	return {"tweet.fields": "created_at"}


def bearer_oauth(r):
	"""
    Method required by bearer token authentication.
    """

	r.headers["Authorization"] = f"Bearer {bearer_token}"
	r.headers["User-Agent"] = "v2UserTweetsPython"
	return r


def connect_to_endpoint(url, params):
	response = requests.request("GET", url, auth=bearer_oauth, params=params)
	print(response.status_code)
	if response.status_code != 200:
		raise Exception(
			"Request returned an error: {} {}".format(
				response.status_code, response.text
			)
		)
	return response.json()

In [143]:
# api limited 100 a times, design a loop to get data, data list from end date to start date, target variable decide when it ends.

def getNeedData():
	url = create_url()
	params = get_params()
	json_response = connect_to_endpoint(url, params)
	one_part_data = pd.DataFrame(json_response["data"])
	part_time_stamp = one_part_data.iloc[-1, 0]
	nowdate = datetime.strptime(part_time_stamp[:10], '%Y-%m-%d')
	# target decide when it stop to grasp the data
	target = datetime(2016, 6, 1, 0, 0)
	while str(nowdate - target)[:1] != '-':
		new_end_time = one_part_data.iloc[-1, 0]
		new_url = create_url(user_id=1339835893, start_time='2016-01-01T00:00:00Z', end_time=part_time_stamp)
		new_params = get_params()
		new_json_response = connect_to_endpoint(new_url, new_params)
		try:
			new_one_part_data = pd.DataFrame(new_json_response["data"])
		except KeyError as e:
			return one_part_data
		part_time_stamp = new_one_part_data.iloc[-1, 0]
		nowdate = datetime.strptime(part_time_stamp[:10], '%Y-%m-%d')

		one_part_data = pd.concat([one_part_data, new_one_part_data])

	return one_part_data


Hillary_2016 = getNeedData()


200
200
200


In [144]:
# I'm sorry for the limited amount of text data from api. I think it might have protection for famous politicians.

Hillary_2016.to_csv(r"/Users/junjiexie/Documents/gu校内生活/501/Assignment1/Hillary_2016_2.csv",encoding="utf-8")

In [ ]:
# test code
#
# url = create_url()
# params = get_params()
# json_response = connect_to_endpoint(url, params)
# tryForSave = json.dumps(json_response, indent=4, sort_keys=True)




# test = pd.DataFrame(json_response["data"])
# time_stamp = test.iloc[-1, 0]
# print(time_stamp[:10])
# test.iloc[-1,0][:10]
# nowdate = datetime.strptime(time_stamp[:10], '%Y-%m-%d')
# target = datetime(2018, 6, 3, 0, 0)
# print(str(nowdate - target)[:1])

